In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
import numpy as np
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
from transformers import AutoProcessor, AutoModel
import torch
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from typing import Dict, List
from src_ubc.classes.survey import Preferences

/Users/kellyzhu/anaconda3/envs/aiha/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Serverless Index

In [10]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY_UBC"))

In [ ]:
# print(pc.list_indexes())

## Initalize Dense DB

In [11]:
# create dense index with integrated embedding 
index_name = "museum-ai-dense"
if not pc.has_index(index_name):
    pc.create_index(index_name, 
                    dimension=512, 
                    metric="cosine", 
                    vector_type="dense",
                    spec=ServerlessSpec( # index deployed region
                        cloud="aws",
                        region="us-east-1"
                    ),
                )
    
dense_db = pc.Index(index_name)

## Initalize Sparse DB

In [12]:
# create sparse index with integrated embedding
index_name = "museum-ai-sparse"
if not pc.has_index(index_name):
    pc.create_index(index_name, 
                    # dimension=512, 
                    metric="dotproduct", 
                    vector_type="sparse",
                    spec=ServerlessSpec(
                        cloud="aws",
                        region="us-east-1"
                    ),
                )

sparse_db = pc.Index(index_name)

## Search Functions

In [ ]:
MODEL_NAME = "BAAI/BGE-VL-base" # or "BAAI/BGE-VL-large"

device = "cuda" if torch.cuda.is_available() else "cpu"
dense_model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True).to(device) # You must set trust_remote_code=True
dense_model_processor = AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)
dense_model.eval()

In [22]:
from PIL import Image

def get_text_embeddings(texts: list[str]) -> list:
    if not texts:
        return []
    inputs = dense_model_processor(text=texts, return_tensors="pt", truncation=True, padding=True).to(device)
    return dense_model.get_text_features(**inputs).cpu()

def get_image_embeddings(image_paths: list[str]) -> list:
    if not image_paths:
        return []
    images = [Image.open(image_path).convert("RGB") for image_path in image_paths]
    inputs = dense_model_processor(images=images, return_tensors="pt").to(device)
    return dense_model.get_image_features(**inputs).cpu()

In [23]:
# Load model directly
sparse_model_tokenizer = AutoTokenizer.from_pretrained("naver/splade-v3")
sparse_model = AutoModelForMaskedLM.from_pretrained("naver/splade-v3").to(device)

In [5]:
def get_sparse_embeddings(texts: list[str]):
    if not texts:
        return []
    tokens = sparse_model_tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = sparse_model(**tokens)
    sparse_embedding = torch.max(torch.log(1 + torch.relu(outputs.logits)) * tokens.attention_mask.unsqueeze(-1), dim=1)[0].detach().cpu()
    
    # convert to pinecone sparse format
    res = []
    for i in range(len(sparse_embedding)):
        indices = sparse_embedding[i].nonzero().squeeze().tolist()
        values = sparse_embedding[i, indices].tolist()
        res.append({"indices": indices, "values": values})
    return res

In [6]:
def merge_chunks(h1, h2, h3):
    """Get the unique hits from two search results and return them as single array of {'_id', 'chunk_text'} dicts, printing each dict on a new line."""
    # Deduplicate by _id
    deduped_hits = {hit['id']: hit for hit in h1['matches'] + h2['matches']}.values()
    # Sort by _score descending
    sorted_hits = sorted(deduped_hits, key=lambda x: x['score'], reverse=True)
    # Transform to format for reranking
    # print(sorted_hits)
    return sorted_hits
    # result = [{'id': hit['id'], 'chunk_text': hit['metadata']['content']} for hit in sorted_hits]
    # return result

def search_with_text(query: str, tags: Preferences):
    dense_results = dense_db.query(
        namespace="umag",
        top_k=5,
        include_values=True,
        include_metadata=True,
        vector=get_text_embeddings([query])[0].tolist()
    ).matches

    sparse_results = sparse_db.query(
        namespace="umag",
        top_k=5,
        include_values=True,
        include_metadata=True,
        sparse_vector=get_sparse_embeddings([query])[0]
    ).matches
    
    tag_results = search_tags(tags)

    merged_results = rrf_merge(dense_results, sparse_results, tag_results)

    return merged_results


def search_with_image(image_path: str):
    dense_results = dense_db.query(
        namespace="umag",
        top_k=20,
        include_values=False,
        include_metadata=True,
        vector=get_image_embeddings([image_path])[0].tolist()
    )

    # sparse_results = sparse_db.query(
    #     namespace="umag",
    #     top_k=5,
    #     include_values=True,
    #     include_metadata=True,
    #     sparse_vector=get_sparse_vector(get_sparse_embeddings([query])[0])
    # )

    # merged_results = merge_chunks(sparse_results, dense_results)

    return dense_results

In [7]:
def get_number_tag_matches(tags: Preferences, hit: Dict) -> float:
    """
    Get the number of tag matches based on how many of the user's tags match the hit's metadata.
    
    Args:
        tags: A Preferences object with attributes:
            - time_period: List[str]
            - themes: List[str]
            - exhibits: List[str]
            - art_medium: List[str]
            - additional_interests: List[str]
        hit: A dictionary representing a search result hit.
        
    Returns:
        A float score representing the number of matching tags.
    """
    score = 0
    for key in tags.__dict__.keys():
        if key in hit['metadata']:
            matches = set(hit['metadata'][key]) & set(getattr(tags, key))
            score += len(matches)
    return score

In [8]:
def search_tags(tags: Preferences, top_k=5):
    """
    Perform a metadata search based on the provided Preferences object.
    
    Args:
        tags: A Preferences object with attributes:
            - time_period: List[str]
            - themes: List[str]
            - exhibits: List[str]
            - art_medium: List[str]
            - additional_interests: List[str]
        top_k: Number of results to return.
        
    Returns:
        List of matching documents from dense_db.
    """
    # Convert the Preferences object to a dictionary.
    tag_dict = {
        "time_period": tags.time_period,
        "themes": tags.themes,
        # "exhibits": tags.exhibits,
        "art_medium": tags.art_medium,
        "additional_interests": tags.additional_interests
    }

    # Remove any keys with empty lists.
    tag_dict = {k: v for k, v in tag_dict.items() if v}
    print(tag_dict)

    # Dummy vector for metadata filtering (using the dimensions expected by your dense index)
    dummy_vector = np.zeros(512).tolist()

 # Build metadata filter conditions. Each condition looks for documents where a given field contains at least one of the values.
    filter_conditions = []
    for key, values in tag_dict.items():
        filter_conditions.append({key: {"$in": values}})
    
    # Use $or operator so that if any condition matches the document is returned
    metadata_filter = {"$or": filter_conditions} if filter_conditions else {}

    # Perform the semantic search using the dummy vector and only filter by metadata.
    response = dense_db.query(
        namespace="umag",
        vector=dummy_vector,
        top_k=top_k,
        include_metadata=True,
        filter=metadata_filter
    )
    
    for i in range(len(response.matches)):
        # normalize the score based on the number of preferences
        response.matches[i]['score'] = get_number_tag_matches(tags, response.matches[i]) / tags.count_preferences()

    return response.matches

In [14]:
tags = Preferences(
    time_period=["ming"],
    themes=["symbolism"],
    exhibits=[],
    art_medium=["ceramics"],  # fixed typo: "cermanics" → "ceramics"
    additional_interests=["Sculpture"]
)

print(tags.count_preferences())  # Output should be 3

test_matches = search_tags(tags)
# test_matches

3
{'time_period': ['ming'], 'themes': ['symbolism'], 'art_medium': ['ceramics'], 'additional_interests': ['Sculpture']}


In [12]:
def calculate_rrf(dense_matches, sparse_matches, tag_matches, k=60):
    """
    Calculate the Reciprocal Rank Fusion (RRF) score for a list of matches.
    
    Args:
        dense_matches: List of dense search results.
        sparse_matches: List of sparse search results.
        tag_matches: List of tag search results.
        k: The k value for RRF calculation.
        
    Returns:
        A list of matches with RRF scores.
    """
    rrf_scores = {}
    
    for match in dense_matches + sparse_matches + tag_matches:
        doc_id = match['id']
        if doc_id not in rrf_scores:
            rrf_scores[doc_id] = 0
        rrf_scores[doc_id] += 1 / (k + match['score'])
    
    sorted_rrf = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    
    return [{'id': doc_id, 'rrf_score': score} for doc_id, score in sorted_rrf]

### Insert Data

In [20]:
import json 
with open("output/Objectifying_China/tagged/en_contents_chunked_sample2_embeddings.json", "r", encoding="utf-8") as f:
        data = json.load(f)

In [ ]:

import json
import itertools

def setup_dense_db(file_path, namespace="umag"):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    vectors = []
    for section_id, chunk in data.items():
        # create vector for text chunks
            vectors.append({
                "id": chunk["id"],
                "values": chunk["dense_text_embedding"],
                "metadata": {
                    "content": chunk["text"],
                    "type": "text",
                    "page_idx": [str(i) for i in chunk["page_idx"]], # convert to string for db
                    "header": chunk["header"],
                    "exhibit": chunk["exhibit"],
                    # TODO: update tags
                    "time_period": chunk['time_period'],
                    "art_medium": chunk['art_medium'],
                    "themes": chunk['themes'],
                    "content_type": chunk["type"],
                }
            })
            # create vector for each image in the chunk
            if "dense_image_embeddings" in chunk:
                for i in range(len(chunk["dense_image_embeddings"])):
                    vectors.append({
                    "id": chunk["id"],
                    "values": chunk["dense_image_embeddings"][i],
                    "metadata": {
                        "type": "image",
                        "page_idx": [str(i) for i in chunk["page_idx"]], # TODO: FIX PAGE IDX SO IT MATCHES THE ORIGINAL PAGE IT WAS ON
                        "imge_path": chunk["img_path"][i]
                    }
            })

    def chunks(iterable, batch_size=200):
        """A helper function to break an iterable into chunks of size batch_size."""
        it = iter(iterable)
        chunk = tuple(itertools.islice(it, batch_size))
        while chunk:
            yield chunk
            chunk = tuple(itertools.islice(it, batch_size))

    for chunk in chunks(vectors, batch_size=200):
        dense_db.upsert(
            vectors=chunk,
            namespace=namespace
        )

    print(f"Inserted {len(vectors)} vectors into the dense index.")

In [7]:
# # delete existing data
# dense_db.delete(delete_all=True, namespace="umag")

{}

In [8]:
setup_dense_db(r"output/Objectifying_China/tagged/en_contents_chunked_sample2_embeddings.json")

Inserted 9 vectors into the dense index.


In [62]:
dense_db.describe_index_stats(namespace="umag")

{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'umag': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}

In [63]:
query = "Jiajing emperor"
dense_results = dense_db.query(
        namespace="umag",
        top_k=5,
        include_values=True,
        include_metadata=True,
        vector=get_text_embeddings([query])[0].tolist()
    )

In [64]:
dense_results

{'matches': [{'id': 'j1m6v08t',
              'metadata': {'art_medium': ['cermanics'],
                           'content': 'With large flaring mouths, vessels of '
                                      'this shape resemble inverted bells and '
                                      'are commonly referred to as ‘chime-type '
                                      'cups’. Popular throughout the Ming and '
                                      'Qing dynasties, this pair was made '
                                      'during the reign of the Jiajing '
                                      "emperor. In keeping with the emperor's "
                                      'interest in auspicious symbolism, the '
                                      'three goats and suns surrounded by '
                                      'prunus, pine and bamboo trees are '
                                      'another example of the use of '
                                      'homophones in the Chine

In [ ]:

def setup_sparse_db(file_path, namespace="umag"):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    vectors = []
    for section_id, chunk in data.items():
        # create vector for text chunks
        vectors.append({
            "id": chunk["id"],
            "sparse_values": chunk["sparse_text_embedding"],
            "metadata": {
                "content": chunk["text"],
                "type": "text",
                "page_idx": [str(i) for i in chunk["page_idx"]], # convert to string for db
                "header": chunk["header"],
                "exhibit": chunk["exhibit"],
                # TODO: update tags
                "time_period": chunk['time_period'],
                "art_medium": chunk['art_medium'],
                "themes": chunk['themes'],
                "content_type": chunk["type"]
                }
            })
        
    def chunks(iterable, batch_size=200):
        """A helper function to break an iterable into chunks of size batch_size."""
        it = iter(iterable)
        chunk = tuple(itertools.islice(it, batch_size))
        while chunk:
            yield chunk
            chunk = tuple(itertools.islice(it, batch_size))


    for chunk in chunks(vectors, batch_size=200):
        sparse_db.upsert(
            vectors=chunk,
            namespace="umag"
        )

    
    print(f"Inserted {len(vectors)} vectors into the sparse index.")
    sparse_db.describe_index_stats(namespace="umag")

In [ ]:
# delete existing vectors
# sparse_db.delete(delete_all=True, namespace="umag")

{}

In [27]:
# setup_sparse_db("output/Objectifying_China/tagged/en_contents_chunked_sample2_embeddings.json")
query = "tell me about Jiajing emperor"
sparse_results = sparse_db.query(
    namespace="umag",
    top_k=5,
    include_values=False,
    include_metadata=True,
    sparse_vector=get_sparse_embeddings([query])[0]
)

print(sparse_results)

{'matches': [{'id': 'j1m6v08t',
              'metadata': {'art_medium': ['cermanics'],
                           'content': 'With large flaring mouths, vessels of '
                                      'this shape resemble inverted bells and '
                                      'are commonly referred to as ‘chime-type '
                                      'cups’. Popular throughout the Ming and '
                                      'Qing dynasties, this pair was made '
                                      'during the reign of the Jiajing '
                                      "emperor. In keeping with the emperor's "
                                      'interest in auspicious symbolism, the '
                                      'three goats and suns surrounded by '
                                      'prunus, pine and bamboo trees are '
                                      'another example of the use of '
                                      'homophones in the Chine

In [ ]:
result = search_with_text('vessels')
result

## Retriever Functions

In [15]:
from collections import defaultdict

def rrf_merge(sparse_results, dense_results, tag_results, k=60):
    def rank_dict(results):
        return {doc['id']: rank for rank, doc in enumerate(results)}

    # Create rank mappings
    sparse_ranks = rank_dict(sparse_results)
    dense_ranks = rank_dict(dense_results)
    tag_ranks = rank_dict(tag_results)

    # Create lookup for originals
    id_to_doc = {}
    for result_set in [sparse_results, dense_results, tag_results]:
        for doc in result_set:
            if doc['id'] not in id_to_doc:
                id_to_doc[doc['id']] = doc

    merged = {}

    # Merge all IDs
    all_ids = set(sparse_ranks) | set(dense_ranks) | set(tag_ranks)
    for id_ in all_ids:
        sparse_rank = sparse_ranks.get(id_)
        dense_rank = dense_ranks.get(id_)
        tag_rank = tag_ranks.get(id_)

        sparse_score = 1 / (k + sparse_rank) if sparse_rank is not None else 0
        dense_score = 1 / (k + dense_rank) if dense_rank is not None else 0
        tag_score = 1 / (k + tag_rank) if tag_rank is not None else 0

        final_score = sparse_score + dense_score + tag_score

        base_doc = id_to_doc[id_]
        merged[id_] = {
            'id': id_,
            'metadata': base_doc['metadata'],
            'sparse_score': sparse_score,
            'dense_score': dense_score,
            'tag_score': tag_score,
            'final_score': final_score,
        }

    return sorted(merged.values(), key=lambda x: x['final_score'], reverse=True)


In [16]:
query = "tell me about the Jiajing emperor"
tags = Preferences(
    time_period=[],
    themes=["symbolism"],
    exhibits=[],
    art_medium=["ceramics"],  # fixed typo: "cermanics" → "ceramics"
    additional_interests=["Sculpture"]
)

sparse_results = sparse_db.query(
    namespace="umag",
    top_k=5,
    include_values=False,
    include_metadata=True,
    sparse_vector=get_sparse_embeddings([query])[0]
)

dense_results = dense_db.query(
    namespace="umag",
    top_k=5,
    include_values=True,
    include_metadata=True,
    vector=get_text_embeddings([query])[0].tolist()
)

tag_results = search_tags(tags)
tag_results

{'themes': ['symbolism'], 'art_medium': ['ceramics'], 'additional_interests': ['Sculpture']}


[{'id': 'j1m6v08t',
  'metadata': {'art_medium': ['cermanics'],
               'content': 'With large flaring mouths, vessels of this shape '
                          'resemble inverted bells and are commonly referred to '
                          'as ‘chime-type cups’. Popular throughout the Ming '
                          'and Qing dynasties, this pair was made during the '
                          'reign of the Jiajing emperor. In keeping with the '
                          "emperor's interest in auspicious symbolism, the "
                          'three goats and suns surrounded by prunus, pine and '
                          'bamboo trees are another example of the use of '
                          'homophones in the Chinese language to create visual '
                          'puns. The goats and suns (both pronounced yang) form '
                          'a rebus that refers to a passage from the I Ching '
                          '(‘The Book of Changes’): “the first 

In [27]:
dense_results.matches

[{'id': 'j1m6v08t',
  'metadata': {'art_medium': ['cermanics'],
               'content': 'With large flaring mouths, vessels of this shape '
                          'resemble inverted bells and are commonly referred to '
                          'as ‘chime-type cups’. Popular throughout the Ming '
                          'and Qing dynasties, this pair was made during the '
                          'reign of the Jiajing emperor. In keeping with the '
                          "emperor's interest in auspicious symbolism, the "
                          'three goats and suns surrounded by prunus, pine and '
                          'bamboo trees are another example of the use of '
                          'homophones in the Chinese language to create visual '
                          'puns. The goats and suns (both pronounced yang) form '
                          'a rebus that refers to a passage from the I Ching '
                          '(‘The Book of Changes’): “the first 

In [ ]:
rrf_merge(dense_results.matches, sparse_results.matches, tag_results)

In [21]:
enumerate(dense_results.matches)

In [18]:
from src_ubc.classes.survey import SurveyResponse, Preferences

response = SurveyResponse(
        major="Computer Science",
        age_group="18-24",
        class_subject="Introduction to AI",
        exhibits=["Exhibit A", "Exhibit B"],
        tour_length_minutes=60,
        time_period=["Modern Art", "Contemporary Art"],
        art_medium=["Digital", "cermanics"],  # fixed typo: "cermanics" → "ceramics"
        themes=["Innovation", "Technology"],
        additional_interests=["Interested in AI applications in art."],
        additional_notes="Looking forward to the tour!"
    )

preferences = Preferences(
        exhibits=response.exhibits,
        time_period=response.time_period,
        art_medium=response.art_medium,
        themes=response.themes,
        additional_interests=response.additional_interests
    )
 
# query = generate_human_query(preferences)

In [15]:
preferences

Preferences(time_period=['Modern Art', 'Contemporary Art'], art_medium=['Digital', 'Sculpture'], themes=['Innovation', 'Technology'], additional_interests=['Interested in AI applications in art.'])

In [19]:
tag_results = search_tags(preferences)


{'time_period': ['Modern Art', 'Contemporary Art'], 'themes': ['Innovation', 'Technology'], 'art_medium': ['Digital', 'cermanics'], 'additional_interests': ['Interested in AI applications in art.']}


In [20]:
tag_results

[{'id': 'j1m6v08t',
  'metadata': {'art_medium': ['cermanics'],
               'content': 'With large flaring mouths, vessels of this shape '
                          'resemble inverted bells and are commonly referred to '
                          'as ‘chime-type cups’. Popular throughout the Ming '
                          'and Qing dynasties, this pair was made during the '
                          'reign of the Jiajing emperor. In keeping with the '
                          "emperor's interest in auspicious symbolism, the "
                          'three goats and suns surrounded by prunus, pine and '
                          'bamboo trees are another example of the use of '
                          'homophones in the Chinese language to create visual '
                          'puns. The goats and suns (both pronounced yang) form '
                          'a rebus that refers to a passage from the I Ching '
                          '(‘The Book of Changes’): “the first 

In [26]:
response = SurveyResponse(
    major="Computer Science",
    age_group="18-24",
    class_subject="Introduction to AI",
    exhibits=["Exhibit A", "Exhibit B"],
    tour_length_minutes=60,
    time_period=["qing"],
    art_medium=["cermanics"],
    themes=[],
    additional_interests=[],
    additional_notes=["Looking forward to the tour!"]
)

preferences = Preferences(
    exhibits=response.exhibits,
    time_period=response.time_period,
    art_medium=response.art_medium,
    themes=response.themes,
    additional_interests=response.additional_interests
)
print(preferences.count_preferences())

4
